# Lab: Vertex AI–Assisted BigQuery Analytics — Example Prompts
**Goal:** Practice moving from simple SQL to complex analytics in BigQuery using *only* carefully engineered prompts with Vertex AI (Gemini).  
**Important:** This notebook contains **prompts only** (no starter code). Paste the prompts into **Vertex AI Studio**, **Vertex AI in Colab Enterprise**, or your chosen chat interface, and then run the generated SQL directly in **BigQuery**. If you decide to automate later, you can ask Vertex AI to convert the winning SQL into a Colab pipeline.

## How to use this prompts-only notebook
1. Open **Vertex AI Studio** (or Gemini in Colab Enterprise chat panel).  
2. Copy a prompt from this notebook and paste it into the model. Do **not** paste any code from here; let the model generate it.  
3. Run the generated SQL in **BigQuery** (Console → BigQuery Studio).  
4. Iterate: refine the prompt when results aren’t what you expect.  
5. Document: capture your final SQL, plus a one-sentence takeaway, in your notes/README.

## Dataset assumptions
Use one of these sources (adjust table paths accordingly):
- **Global Superstore (Kaggle)** loaded into BigQuery (e.g., `[YOUR_PROJECT].superstore_data.sales`)  
- **TheLook eCommerce** public dataset: `bigquery-public-data.thelook_ecommerce`  
If you are using *Global Superstore*, make sure column names match your schema (e.g., `Order_Date`, `Region`, `Category`, `Sub_Category`, `Sales`, `Profit`, `Discount`, `State`, `Customer_ID`, `Ship_Mode`).

---
## Prompting guardrails (quick checklist)
- **Be explicit**: table path, column names, filters, output columns, sort order, and limits.  
- **Ask for runnable SQL**: “Return a BigQuery SQL block only.”  
- **Control cost**: ask for `LIMIT` during exploration and remove it for the final run.  
- **Validate**: request a brief explanation of why each clause is present and how you can sanity-check results.
---

## Install Dependencies

In [25]:
# Install the Google Cloud BigQuery client library
!pip install google-cloud-bigquery==3.17.0 pandas==2.1.4

# Authenticate your Colab environment
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Copy Schema to a dataframe

In [26]:
from google.cloud import bigquery
import pandas as pd

# Replace with your Google Cloud Project ID
project_id = 'original-wonder-471819-n2' # This is derived from your provided table name
dataset_id = 'lab1_foundation'
table_id = 'Superstore'

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

# Get the table object
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)

# Extract schema information
schema_list = []
for field in table.schema:
    schema_list.append({
        'name': field.name,
        'field_type': field.field_type,
        'mode': field.mode,
        'description': field.description
    })

# Convert to Pandas DataFrame
schema_df = pd.DataFrame(schema_list)

# Display the schema DataFrame (optional, for verification)
print("Schema DataFrame created:")
# To see the output, run the code.


Schema DataFrame created:


## CLean Column Names

In [27]:
# --- 1. Clean the Column Names ---
# Create a 'clean_name' column with standard naming conventions:
# lowercase, with spaces and hyphens replaced by underscores.
schema_df['clean_name'] = schema_df['name'].str.lower().str.replace(' ', '_').str.replace('-', '_')


# --- 2. Generate the Aliases for the SELECT Clause ---
column_expressions = []
for index, row in schema_df.iterrows():
    original_name = row['name']
    clean_name = row['clean_name']

    # If the original name contains a space or special character, it needs to be
    # enclosed in backticks (`) in the SQL statement.
    if ' ' in original_name or '-' in original_name:
        expression = f'`{original_name}` AS {clean_name}'
    else:
        # If the name is already clean, we still alias it for consistency.
        expression = f'{original_name} AS {clean_name}'
    column_expressions.append(expression)

# Join all the individual column expressions into a single, formatted string.
select_clause = ",\n  ".join(column_expressions)


# --- 3. Construct the Final CREATE VIEW Statement ---
new_view_id = 'superstore_clean' # You can change this if you like

create_view_sql = f"""
CREATE OR REPLACE VIEW `{project_id}.{dataset_id}.{new_view_id}` AS
SELECT
  {select_clause}
FROM
  `{project_id}.{dataset_id}.{table_id}`;
"""

# --- 4. Print the Final SQL ---
print("--- Copy the SQL below and run it in your BigQuery Console ---")
print(create_view_sql)

--- Copy the SQL below and run it in your BigQuery Console ---

CREATE OR REPLACE VIEW `original-wonder-471819-n2.lab1_foundation.superstore_clean` AS
SELECT
  `Row ID` AS row_id,
  `Order ID` AS order_id,
  `Order Date` AS order_date,
  `Ship Date` AS ship_date,
  `Ship Mode` AS ship_mode,
  `Customer ID` AS customer_id,
  `Customer Name` AS customer_name,
  Segment AS segment,
  Country AS country,
  City AS city,
  State AS state,
  `Postal Code` AS postal_code,
  Region AS region,
  `Product ID` AS product_id,
  Category AS category,
  `Sub-Category` AS sub_category,
  `Product Name` AS product_name,
  Sales AS sales,
  Quantity AS quantity,
  Discount AS discount,
  Profit AS profit
FROM
  `original-wonder-471819-n2.lab1_foundation.Superstore`;



## Generate View with standard column naming convention

In [28]:
# Execute the CREATE VIEW SQL query
try:
    query_job = client.query(create_view_sql)  # API request
    query_job.result()  # Waits for the query to finish
    print(f"View '{new_view_id}' created/replaced successfully in dataset '{dataset_id}'.")
except Exception as e:
    print(f"An error occurred while creating the view: {e}")

# Now, let's print 10 rows from the newly created view to verify
print(f"\n--- First 10 rows from the new view '{new_view_id}' ---")
try:
    # Construct a reference to the new view
    view_table_ref = client.dataset(dataset_id).table(new_view_id)

    # Fetch the first 10 rows
    rows = client.list_rows(view_table_ref, max_results=10)

    # Print header
    print(" | ".join([field.name for field in rows.schema]))
    print("-" * 80) # Separator

    # Print rows
    for row in rows:
        print(" | ".join([str(item) for item in row.values()]))

except Exception as e:
    print(f"An error occurred while fetching rows from the view: {e}")



View 'superstore_clean' created/replaced successfully in dataset 'lab1_foundation'.

--- First 10 rows from the new view 'superstore_clean' ---
row_id | order_id | order_date | ship_date | ship_mode | customer_id | customer_name | segment | country | city | state | postal_code | region | product_id | category | sub_category | product_name | sales | quantity | discount | profit
--------------------------------------------------------------------------------
An error occurred while fetching rows from the view: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/original-wonder-471819-n2/datasets/lab1_foundation/tables/superstore_clean/data?maxResults=10&formatOptions.useInt64Timestamp=True&prettyPrint=false: Cannot list a table of type VIEW.


In [29]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  order_id,
  customer_name,
  product_name,
  sales,
  profit
FROM
  `original-wonder-471819-n2.lab1_foundation.superstore_clean`
LIMIT 10;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and the original table has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 10 rows.

--- Displaying Results ---


,order_id,customer_name,product_name,sales,profit
0,CA-2015-154900,Sung Shariari,Avery 518,3.15,1.5120
1,CA-2015-154900,Sung Shariari,Adams Telephone Message Book W/Dividers/Space ...,22.72,10.2240
2,US-2016-152415,Patrick O'Donnell,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,6.2244
3,US-2016-152415,Patrick O'Donnell,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,61.3824
4,CA-2016-153269,Pamela Stobb,"Personal Folder Holder, Ebony",11.21,3.3630
5,CA-2016-153269,Pamela Stobb,"Situations Contoured Folding Chairs, 4/Set",354.90,88.7250
6,CA-2016-153269,Pamela Stobb,Xerox 193,17.94,8.7906
7,CA-2016-153269,Pamela Stobb,GBC Binding covers,51.80,23.3100
8,CA-2015-158792,Brian Dahlen,Staples,22.20,10.4340
9,CA-2016-141082,Fred McMath,Avery 517,3.69,1.7343


## Part A — SQL Warm‑Up (SELECT, WHERE, ORDER BY, LIMIT, DISTINCT)
**Aim:** Build confidence with precise, unambiguous prompts that yield clean, runnable SQL.

### A1. Unique values (DISTINCT)
**Prompt (paste in Vertex AI):**
```
Act as a senior BigQuery analyst. Produce a **single runnable BigQuery SQL** (no commentary) for:
- Task: List all unique `Sub_Category` values sold in the 'West' region.
- Table: `original-wonder-471819-n2.lab1_foundation.superstore_clean`
- Filter: `Region = 'West'`
- Output: a single column named `Sub_Category`
- Sort: alphabetically A→Z
- Add: `LIMIT 100` to control cost during exploration.
```
**Reflection:** Did the result match your expectations? If not, what ambiguity in your prompt might have caused the mismatch?

In [30]:
# prompt: Act as a senior BigQuery analyst. Produce a **single runnable BigQuery SQL** (no commentary) for:
# - Task: List all unique `Sub_Category` values sold in the 'West' region.
# - Table: `original-wonder-471819-n2.lab1_foundation.superstore_clean`
# - Filter: `Region = 'West'`
# - Output: a single column named `Sub_Category`
# - Sort: alphabetically A→Z
# - Add: `LIMIT 100` to control cost during exploration.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT DISTINCT
  Sub_Category
FROM
  `original-wonder-471819-n2.lab1_foundation.superstore_clean`
WHERE
  Region = 'West'
ORDER BY
  Sub_Category ASC
LIMIT 100;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and the original table has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")


✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 17 rows.

--- Displaying Results ---


,Sub_Category
0,Accessories
1,Appliances
2,Art
3,Binders
4,Bookcases
5,Chairs
6,Copiers
7,Envelopes
8,Fasteners
9,Furnishings


### A2. Top‑N by metric (ORDER BY … DESC)
**Prompt:**
```
BigQuery SQL only.
Task: Return the top 10 customers by total profit.
Table: `mgmt-467-47888.lab_foundation.superstore`
Columns used: `Customer_ID`, `Profit`
Output columns: `Customer_ID`, `total_profit`
Logic: SUM Profit per customer, order by `total_profit` DESC
Add `LIMIT 10`.
```
**Tip:** If your schema uses different identifiers (e.g., `Customer Name`), restate column names explicitly.

In [31]:
# prompt: BigQuery SQL only.
# Task: Return the top 10 customers by total profit.
# Table: `mgmt-467-47888.lab_foundation.superstore`
# Columns used: `Customer_ID`, `Profit`
# Output columns: `Customer_ID`, `total_profit`
# Logic: SUM Profit per customer, order by `total_profit` DESC
# Add `LIMIT 10`.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  Customer_ID,
  SUM(Profit) AS total_profit
FROM
  `original-wonder-471819-n2.lab1_foundation.superstore_clean`
GROUP BY
  Customer_ID
ORDER BY
  total_profit DESC
LIMIT 10;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your table exists and has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 10 rows.

--- Displaying Results ---


,Customer_ID,total_profit
0,TC-20980,8981.3239
1,RB-19360,6976.0959
2,SC-20095,5757.4119
3,HL-15040,5622.4292
4,AB-10105,5444.8055
5,TA-21385,4703.7883
6,CM-12385,3899.8904
7,KD-16495,3038.6254
8,AR-10540,2884.6208
9,DR-12940,2869.0760


### A3. Basic filtering (WHERE) + sanity checks
**Prompt:**
```
BigQuery SQL only.
Task: Count orders shipped with each `Ship_Mode`, but only for orders in the 'Technology' category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Ship_Mode`, `order_count`
Logic: COUNT(*) grouped by `Ship_Mode`
Sort by `order_count` DESC
```
**Validation ask:** “Also list two quick sanity checks to verify the numbers.”

In [32]:
# prompt: BigQuery SQL only.
# Task: Count orders shipped with each `Ship_Mode`, but only for orders in the 'Technology' category.
# Table: `[YOUR_PROJECT].superstore_data.sales`
# Output: `Ship_Mode`, `order_count`
# Logic: COUNT(*) grouped by `Ship_Mode`
# Sort by `order_count` DESC

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  Ship_Mode,
  COUNT(*) AS order_count
FROM
  `original-wonder-471819-n2.lab1_foundation.superstore_clean`
WHERE
  Category = 'Technology'
GROUP BY
  Ship_Mode
ORDER BY
  order_count DESC;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your table exists and has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

    print("\n--- Sanity Checks ---")
    print("1. Manually check the count for one Ship_Mode in the BigQuery UI for the 'Technology' category.")
    print("2. Verify that the sum of 'order_count' across all Ship_Modes equals the total number of orders in the 'Technology' category.")

except Exception as e:
    print(f"\n❌ An error occurred: {e}")


✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 4 rows.

--- Displaying Results ---


,Ship_Mode,order_count
0,Standard Class,1082
1,Second Class,366
2,First Class,301
3,Same Day,98



--- Sanity Checks ---
1. Manually check the count for one Ship_Mode in the BigQuery UI for the 'Technology' category.
2. Verify that the sum of 'order_count' across all Ship_Modes equals the total number of orders in the 'Technology' category.


## Part B — Grouped Analytics (GROUP BY, HAVING)
**Aim:** Turn raw facts into grouped metrics and filtered aggregations.

### B1. KPI aggregation with WHERE + GROUP BY
**Prompt:**
```
BigQuery SQL only.
Task: Compute monthly revenue for the last 12 full months.
Table: `[YOUR_PROJECT].superstore_data.sales`
Assume: `Order_Date` is a DATE or TIMESTAMP column named exactly `Order_Date`.
Output: `year_month` (YYYY-MM format), `monthly_revenue`
Logic: Truncate date to month, SUM `Sales`, filter to last 12 full months.
Sort by `year_month` ascending.
Include a `LIMIT` safeguard for exploration.
```

In [43]:
# prompt: BigQuery SQL only.
# Task: Compute monthly revenue for the last 12 full months.
# Table: `[YOUR_PROJECT].superstore_data.sales`
# Assume: `Order_Date` is a DATE or TIMESTAMP column named exactly `Order_Date`.
# Output: `year_month` (YYYY-MM format), `monthly_revenue`
# Logic: Truncate date to month, SUM `Sales`, filter to last 12 full months.
# Sort by `year_month` ascending.
# Include a `LIMIT` safeguard for exploration.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  FORMAT_DATE('%Y-%m', Order_Date) AS year_month,
  SUM(Sales) AS monthly_revenue
FROM
  `original-wonder-471819-n2.lab1_foundation.superstore_clean`
-- Instead of current date, anchor to the max Order_Date in the dataset
WHERE
  Order_Date >= DATE_SUB(DATE_TRUNC((SELECT MAX(Order_Date) FROM `original-wonder-471819-n2.lab1_foundation.superstore_clean`), MONTH), INTERVAL 12 MONTH)
  AND Order_Date < DATE_TRUNC((SELECT MAX(Order_Date) FROM `original-wonder-471819-n2.lab1_foundation.superstore_clean`), MONTH)
GROUP BY
  year_month
ORDER BY
  year_month ASC
LIMIT 100;
"""


print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your table exists and has data, and that the date range is appropriate for your data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")


✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 12 rows.

--- Displaying Results ---


,year_month,monthly_revenue
0,2016-12,96999.0430
1,2017-01,43971.3740
2,2017-02,20301.1334
3,2017-03,58872.3528
4,2017-04,36521.5361
5,2017-05,44261.1102
6,2017-06,52981.7257
7,2017-07,45264.4160
8,2017-08,63120.8880
9,2017-09,87866.6520


### B2. Post‑aggregation filter (HAVING)
**Prompt:**
```
BigQuery SQL only.
Task: Find sub-categories whose total profit over the entire dataset is negative.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Sub_Category`, `total_profit`
Logic: SUM `Profit` GROUP BY `Sub_Category`, HAVING SUM(Profit) < 0
Sort by `total_profit` ASC (most negative first).
```
**Why HAVING?** Ask the model to include a 1-sentence explanation of why HAVING is used instead of WHERE here.

In [34]:
# prompt: BigQuery SQL only.
# Task: Find sub-categories whose total profit over the entire dataset is negative.
# Table: `[YOUR_PROJECT].superstore_data.sales`
# Output: `Sub_Category`, `total_profit`
# Logic: SUM `Profit` GROUP BY `Sub_Category`, HAVING SUM(Profit) < 0
# Sort by `total_profit` ASC (most negative first).

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  Sub_Category,
  SUM(Profit) AS total_profit
FROM
  `original-wonder-471819-n2.lab1_foundation.superstore_clean`
GROUP BY
  Sub_Category
HAVING
  SUM(Profit) < 0
ORDER BY
  total_profit ASC;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. This might mean there are no sub-categories with negative total profit in your dataset.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

    print("\n--- Explanation of HAVING ---")
    print("HAVING is used instead of WHERE because it filters groups based on an aggregated value (SUM(Profit) in this case), whereas WHERE filters individual rows *before* aggregation.")

except Exception as e:
    print(f"\n❌ An error occurred: {e}")


✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 3 rows.

--- Displaying Results ---


,Sub_Category,total_profit
0,Tables,-17725.4811
1,Bookcases,-3472.5560
2,Supplies,-1189.0995



--- Explanation of HAVING ---
HAVING is used instead of WHERE because it filters groups based on an aggregated value (SUM(Profit) in this case), whereas WHERE filters individual rows *before* aggregation.


## Part C — Joins (dimension enrichment)
**Aim:** Use joins to enhance facts with attributes.

### C1. Join facts to a small dimension
*(If you have a customer or product dimension in your schema, use it. Otherwise, request a synthetic example.)*  
**Prompt:**
```
BigQuery SQL only.
Task: Join the sales table to a product dimension to report `Product_ID`, `Product_Name`, and total sales.
Tables: `[YOUR_PROJECT].superstore_data.sales` as s, `[YOUR_PROJECT].superstore_data.products` as p
Join key: `s.Product_ID = p.Product_ID`
Output: `Product_ID`, `Product_Name`, `total_sales`
Sort by `total_sales` DESC
```
**If you lack a dimension table:** Ask the model how to simulate one temporarily via a CTE.

In [35]:
# prompt: (If you have a customer or product dimension in your schema, use it. Otherwise, request a synthetic example.)
# Prompt:
# BigQuery SQL only.
# Task: Join the sales table to a product dimension to report `Product_ID`, `Product_Name`, and total sales.
# Tables: `[YOUR_PROJECT].superstore_data.sales` as s, `[YOUR_PROJECT].superstore_data.products` as p
# Join key: `s.Product_ID = p.Product_ID`
# Output: `Product_ID`, `Product_Name`, `total_sales`
# Sort by `total_sales` DESC
# If you lack a dimension table: Ask the model how to simulate one temporarily via a CTE.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  s.Product_ID,
  p.Product_Name,
  SUM(s.Sales) AS total_sales
FROM
  `original-wonder-471819-n2.lab1_foundation.superstore_clean` AS s
JOIN
  `original-wonder-471819-n2.lab1_foundation.superstore_clean` AS p ON s.Product_ID = p.Product_ID
GROUP BY
  s.Product_ID, p.Product_Name
ORDER BY
  total_sales DESC
LIMIT 100;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your tables exist, have data, and that the join key is correct.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")


✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 100 rows.

--- Displaying Results ---


,Product_ID,Product_Name,total_sales
0,TEC-CO-10004722,Canon imageCLASS 2200 Advanced Copier,307999.120
1,OFF-BI-10003527,Fellowes PB500 Electric Punch Plastic Comb Bin...,274533.840
2,OFF-BI-10001359,GBC DocuBind TL300 Electric Binding System,218058.269
3,FUR-CH-10002024,HON 5400 Series Task Chairs for Big and Tall,174964.608
4,OFF-BI-10000545,GBC Ibimaster 500 Manual ProClick Binding System,171220.500
...,...,...,...
95,OFF-ST-10000736,Carina Double Wide Media Storage Towers in Nat...,36343.824
96,OFF-ST-10000046,Fellowes Super Stor/Drawer Files,36057.960
97,TEC-PH-10001557,Pyle PMP37LED,35900.260
98,TEC-AC-10001838,Razer Tiamat Over Ear 7.1 Surround Sound PC Ga...,35838.208


## Part D — Common Table Expressions (CTEs)
**Aim:** Make complex logic readable and testable in steps.

### D1. Multi‑step ranking with CTEs
**Prompt:**
```
BigQuery SQL only.
Goal: Within each `Region`, rank states by total sales and return top 3 per region.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE 1 (`state_sales`): SUM(Sales) by `Region`, `State`
CTE 2 (`ranked_state_sales`): Add `RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as `sales_rank`
Final SELECT: rows where `sales_rank <= 3`
Output columns: `Region`, `State`, `total_sales`, `sales_rank`
Sort: by `Region`, then `sales_rank`
```
**Ask for**: a one-paragraph explanation of each step, then **provide only the final runnable SQL**.

In [36]:
# prompt: Develope the code in BigQuery SQL only using python.
# Goal: Within each `Region`, rank states by total sales and return top 3 per region.
# Table: `[YOUR_PROJECT].superstore_data.sales`
# CTE 1 (`state_sales`): SUM(Sales) by `Region`, `State`
# CTE 2 (`ranked_state_sales`): Add `RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as `sales_rank`
# Final SELECT: rows where `sales_rank <= 3`
# Output columns: `Region`, `State`, `total_sales`, `sales_rank`
# Sort: by `Region`, then `sales_rank`
# Provide a one-paragraph explanation of each step, then provide only the final runnable SQL with the explanation within the code as comments.

print("✅ Step 1: Defining the query string with CTEs for ranking states by sales within each region...")

query_string = """
-- CTE 1: Calculate total sales for each state within each region.
-- This CTE aggregates the sales data to provide a sum of sales for every
-- unique combination of Region and State.
WITH state_sales AS (
  SELECT
    Region,
    State,
    SUM(Sales) AS total_sales
  FROM
    `original-wonder-471819-n2.lab1_foundation.superstore_clean`
  GROUP BY
    Region,
    State
),

-- CTE 2: Rank states by total sales within each region.
-- This CTE uses the RANK() window function to assign a rank to each state
-- based on its total sales in descending order. The ranking is partitioned
-- by Region, ensuring that ranks are calculated independently for each region.
ranked_state_sales AS (
  SELECT
    Region,
    State,
    total_sales,
    RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC) AS sales_rank
  FROM
    state_sales
)

-- Final SELECT: Retrieve the top 3 states by sales rank for each region.
-- This query selects rows from the ranked_state_sales CTE where the sales_rank
-- is less than or equal to 3, effectively filtering for the top 3 states
-- in each region. The results are then ordered by Region and sales_rank.
SELECT
  Region,
  State,
  total_sales,
  sales_rank
FROM
  ranked_state_sales
WHERE
  sales_rank <= 3
ORDER BY
  Region ASC,
  sales_rank ASC;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your table exists and has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string with CTEs for ranking states by sales within each region...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 12 rows.

--- Displaying Results ---


,Region,State,total_sales,sales_rank
0,Central,Texas,170188.0458,1
1,Central,Illinois,80166.1010,2
2,Central,Michigan,76269.6140,3
3,East,New York,310876.2710,1
4,East,Pennsylvania,116511.9140,2
5,East,Ohio,78258.1360,3
6,South,Florida,89473.7080,1
7,South,Virginia,70636.7200,2
8,South,North Carolina,55603.1640,3
9,West,California,457687.6315,1


### D2. Time‑boxed “most improved” analysis
**Prompt:**
```
BigQuery SQL only.
Goal: Identify the top 5 sub-categories with the largest YoY revenue increase from 2023 to 2024.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `yr_sales`: SUM(Sales) by `Sub_Category` and `year` extracted from `Order_Date`
Final: pivot or self-join to compute delta (2024 minus 2023) as `yoy_delta`
Output: `Sub_Category`, `sales_2023`, `sales_2024`, `yoy_delta`
Order by `yoy_delta` DESC
Limit 5
```
**Validation:** Ask the model for two quick failure modes (e.g., missing years) and how to handle them.

Generate the code in BigQuery SQL only while using python.
Goal: Identify the top 5 sub-categories with the largest YoY revenue increase from 2023 to 2024.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `yr_sales`: SUM(Sales) by `Sub_Category` and `year` extracted from `Order_Date`
Final: pivot or self-join to compute delta (2024 minus 2023) as `yoy_delta`
Output: `Sub_Category`, `sales_2023`, `sales_2024`, `yoy_delta`
Order by `yoy_delta` DESC
Limit 5

I would then like you to provide two quick failure modes (e.g., missing years) and within the code how to handle them.

In [45]:
# prompt: Generate the code in BigQuery SQL only while using python.
# Goal: Identify the top 5 sub-categories with the largest YoY revenue increase from the most recent two years in the dataset.
# Table: `[YOUR_PROJECT].superstore_data.sales`
# CTE `yr_sales`: SUM(Sales) by `Sub_Category` and `year` extracted from `Order_Date`
# Final: pivot/self-join to compute delta (latest_year minus prev_year) as `yoy_delta`
# Output: `Sub_Category`, `sales_prev_year`, `sales_curr_year`, `yoy_delta`
# Order by `yoy_delta` DESC
# Limit 5
# Failure modes handled:
#   1) Missing year for a sub-category -> COALESCE to 0 so the delta is well-defined.
#   2) Dataset not having 2 full years -> still returns whatever exists; filters out all-zero rows.

print("✅ Step 1: Defining the query string...")

query_string = """
-- Determine the most recent year present in the data,
-- then compare it to the previous year (dynamic, no hard-coding 2023/2024).
WITH bounds AS (
  SELECT
    EXTRACT(YEAR FROM MAX(Order_Date)) AS max_year
  FROM `original-wonder-471819-n2.lab1_foundation.superstore_clean`
),
yr_sales AS (
  SELECT
    Sub_Category,
    EXTRACT(YEAR FROM Order_Date) AS year,
    SUM(Sales) AS total_sales
  FROM
    `original-wonder-471819-n2.lab1_foundation.superstore_clean`
  WHERE
    EXTRACT(YEAR FROM Order_Date) IN (
      (SELECT max_year - 1 FROM bounds),
      (SELECT max_year     FROM bounds)
    )
  GROUP BY
    Sub_Category, year
),
pivoted_sales AS (
  SELECT
    Sub_Category,
    -- Failure mode #1 (missing year): use COALESCE so missing years become 0
    COALESCE(SUM(CASE WHEN year = (SELECT max_year - 1 FROM bounds) THEN total_sales END), 0) AS sales_prev_year,
    COALESCE(SUM(CASE WHEN year = (SELECT max_year     FROM bounds) THEN total_sales END), 0) AS sales_curr_year
  FROM yr_sales
  GROUP BY Sub_Category
)
SELECT
  Sub_Category,
  sales_prev_year AS sales_2023,   -- label stays as-is per original prompt format
  sales_curr_year AS sales_2024,   -- these correspond to prev/curr dynamic years
  (sales_curr_year - sales_prev_year) AS yoy_delta
FROM pivoted_sales
-- Failure mode #2 (sparse/no data): drop rows with zero in both years
WHERE (sales_prev_year + sales_curr_year) > 0
ORDER BY yoy_delta DESC
LIMIT 5;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. "
              "Check that the table has at least one year of data; this query compares the latest year it finds to the prior year.")
    else:
        print("\n--- Top 5 Sub-Categories by YoY Revenue Increase (dynamic last two years) ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")


print("\n--- Failure Modes and Handling ---")
print("1. Missing Years: If a Sub_Category exists in 2023 but not 2024 (or vice-versa), the `CASE` statements in `pivoted_sales` will correctly assign 0 to the missing year's sales. The `yoy_delta` calculation will then accurately reflect the change (e.g., a decrease if it was present in 2023 but not 2024). The `WHERE sales_2023 > 0 OR sales_2024 > 0` clause ensures we don't include sub-categories that had no sales in either year.")
print("2. No YoY Change: If a Sub_Category had the same sales in both 2023 and 2024, `yoy_delta` will be 0. These will be included in the results but will appear lower in the sorted list unless other sub-categories also have a 0 delta.")


✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 5 rows.

--- Top 5 Sub-Categories by YoY Revenue Increase (dynamic last two years) ---


,Sub_Category,sales_2023,sales_2024,yoy_delta
0,Phones,78962.030,105340.516,26378.486
1,Binders,49683.325,72788.045,23104.720
2,Accessories,41895.854,59946.232,18050.378
3,Appliances,26050.315,42926.932,16876.617
4,Copiers,49599.410,62899.388,13299.978



--- Failure Modes and Handling ---
1. Missing Years: If a Sub_Category exists in 2023 but not 2024 (or vice-versa), the `CASE` statements in `pivoted_sales` will correctly assign 0 to the missing year's sales. The `yoy_delta` calculation will then accurately reflect the change (e.g., a decrease if it was present in 2023 but not 2024). The `WHERE sales_2023 > 0 OR sales_2024 > 0` clause ensures we don't include sub-categories that had no sales in either year.
2. No YoY Change: If a Sub_Category had the same sales in both 2023 and 2024, `yoy_delta` will be 0. These will be included in the results but will appear lower in the sorted list unless other sub-categories also have a 0 delta.


## Part E — Window Functions (ROW_NUMBER, RANK, DENSE_RANK, LAG/LEAD, moving averages)
**Aim:** Compare rows across partitions and time; compute trends and ranks without collapsing rows.

### E1. Top product per region (ROW_NUMBER)
**Prompt:**
```
BigQuery SQL only.
Task: For each `Region`, return only the single highest-revenue `Sub_Category`.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `subcat_sales`: SUM(Sales) by `Region`, `Sub_Category`
Add `ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as rn
Final: filter `rn = 1`
Output: `Region`, `Sub_Category`, `total_sales`
Sort by `Region`
```
**Why `ROW_NUMBER` instead of `RANK`?** Ask the model to add a 2-sentence contrast.

In [38]:
# prompt: BigQuery SQL only.
# Task: For each `Region`, return only the single highest-revenue `Sub_Category`.
# Table: `[YOUR_PROJECT].superstore_data.sales`
# CTE `subcat_sales`: SUM(Sales) by `Region`, `Sub_Category`
# Add `ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as rn
# Final: filter `rn = 1`
# Output: `Region`, `Sub_Category`, `total_sales`
# Sort by `Region`
# At the end of the code, in comments I would like you to generate a 2 sentence contrast asking the question "Why ROW_NUMBER instead of RANK?"

print("✅ Step 1: Defining the query string with CTE for sub-category sales per region and ranking...")

query_string = """
WITH subcat_sales AS (
  SELECT
    Region,
    Sub_Category,
    SUM(Sales) AS total_sales
  FROM
    `original-wonder-471819-n2.lab1_foundation.superstore_clean`
  GROUP BY
    Region,
    Sub_Category
),
ranked_subcat_sales AS (
  SELECT
    Region,
    Sub_Category,
    total_sales,
    ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC) AS rn
  FROM
    subcat_sales
)
SELECT
  Region,
  Sub_Category,
  total_sales
FROM
  ranked_subcat_sales
WHERE
  rn = 1
ORDER BY
  Region ASC;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your table exists and has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

# Why ROW_NUMBER instead of RANK?
# ROW_NUMBER assigns a unique sequential integer to each row within a partition, ensuring only one row is selected even if there are ties in total_sales. RANK, on the other hand, would assign the same rank to tied rows, potentially returning multiple sub-categories if they have identical highest sales within a region.


✅ Step 1: Defining the query string with CTE for sub-category sales per region and ranking...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 4 rows.

--- Displaying Results ---


,Region,Sub_Category,total_sales
0,Central,Chairs,85230.646
1,East,Phones,100614.982
2,South,Phones,58304.438
3,West,Chairs,101781.328


### E2. YoY growth with LAG
**Prompt:**
```
BigQuery SQL only.
Task: Compute year-over-year revenue growth for 'Phones' sub-category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Filter to `Sub_Category = 'Phones'`
- Aggregate yearly revenue using EXTRACT(YEAR FROM Order_Date)
- Add `LAG(yearly_revenue) OVER (ORDER BY year)` as `prev_revenue`
- Compute `yoy_pct = 100.0 * (yearly_revenue - prev_revenue) / prev_revenue`
Output: `year`, `yearly_revenue`, `prev_revenue`, `yoy_pct`
Sort by `year` ASC
```
**Ask for**: a guard against divide-by-zero or NULL previous year.

In [49]:
# prompt: Generate using BigQuery SQL only while coding in python.
# Task: Compute year-over-year revenue growth for 'Phones' sub-category.
# Guard: Use SAFE_DIVIDE to avoid divide-by-zero/NULL.

print("✅ Step 1: Defining the query string for YoY growth calculation...")

query_string = """
WITH yr_rev AS (
  SELECT
    EXTRACT(YEAR FROM Order_Date) AS yr,
    SUM(Sales) AS yearly_revenue
  FROM
    `original-wonder-471819-n2.lab1_foundation.superstore_clean`
  WHERE
    Sub_Category = 'Phones'
  GROUP BY
    yr
),
lagged AS (
  SELECT
    yr AS year,
    yearly_revenue,
    LAG(yearly_revenue) OVER (ORDER BY yr) AS prev_revenue
  FROM
    yr_rev
)
SELECT
  year,
  yearly_revenue,
  prev_revenue,
  SAFE_DIVIDE(yearly_revenue - prev_revenue, prev_revenue) * 100 AS yoy_pct
FROM
  lagged
ORDER BY
  year ASC;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. "
              "Verify that 'Phones' exists in the dataset and spans at least two years.")
    else:
        print("\n--- 'Phones' Year-over-Year Revenue Growth ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")


✅ Step 1: Defining the query string for YoY growth calculation...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 4 rows.

--- 'Phones' Year-over-Year Revenue Growth ---


,year,yearly_revenue,prev_revenue,yoy_pct
0,2014,77390.806,NaN,NaN
1,2015,68313.702,77390.806,-11.728918
2,2016,78962.030,68313.702,15.587397
3,2017,105340.516,78962.030,33.406545


### E3. 3‑month moving average (MA)
**Prompt:**
```
BigQuery SQL only.
Task: For the 'Corporate' segment, compute a 3-month moving average of monthly revenue.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Derive `month` via DATE_TRUNC(Order_Date, MONTH)
- SUM(Sales) per `month`
- Add `AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)` as `ma_3`
Output: `month`, `monthly_revenue`, `ma_3`
Sort by `month` ASC
```
**Tip:** Ask the model to include a 1‑line cost control note (e.g., restrict date range while iterating).

In [51]:
# prompt: BigQuery SQL only.
# Task: For the 'Corporate' segment, compute a 3-month moving average of monthly revenue.
# Table: `[YOUR_PROJECT].superstore_data.sales`
# Steps:
# - Derive `month` via DATE_TRUNC(Order_Date, MONTH)
# - SUM(Sales) per `month`
# - Add `AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)` as `ma_3`
# Output: `month`, `monthly_revenue`, `ma_3`
# Sort by `month` ASC
# IMPORTANT: Include a 1-line cost control note (e.g., restrict date range while iterating). Include this in the comments of the code

print("✅ Step 1: Defining the query string for 3-month moving average calculation...")

query_string = """
-- Cost control note: While iterating, limit scanned data (e.g., add WHERE Order_Date BETWEEN '2014-01-01' AND '2014-12-31').
WITH monthly_revenue AS (
  SELECT
    DATE_TRUNC(Order_Date, MONTH) AS month,
    SUM(Sales) AS monthly_revenue
  FROM
    `original-wonder-471819-n2.lab1_foundation.superstore_clean`
  WHERE
    Segment = 'Corporate'
  GROUP BY
    month
)
SELECT
  m.month,
  m.monthly_revenue,
  AVG(m.monthly_revenue) OVER (
    ORDER BY m.month
    ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
  ) AS ma_3
FROM
  monthly_revenue AS m
ORDER BY
  m.month ASC;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. "
              "Confirm that the 'Corporate' segment exists and that date filters (if any) match your data range.")
    else:
        print("\n--- 3-Month Moving Average of Monthly Revenue for 'Corporate' Segment ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")


✅ Step 1: Defining the query string for 3-month moving average calculation...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 48 rows.

--- 3-Month Moving Average of Monthly Revenue for 'Corporate' Segment ---


,month,monthly_revenue,ma_3
0,2014-01-01,1701.5280,1701.528000
1,2014-02-01,1183.6680,1442.598000
2,2014-03-01,11106.7990,4663.998333
3,2014-04-01,14131.7290,8807.398667
4,2014-05-01,9142.0000,11460.176000
5,2014-06-01,3970.9140,9081.547667
6,2014-07-01,10032.9880,7715.300667
7,2014-08-01,7451.7740,7151.892000
8,2014-09-01,15507.7450,10997.502333
9,2014-10-01,12637.6780,11865.732333


## Part F — Debugging & Optimization Prompts
**Aim:** Use the model as a rubber duck for error handling and performance.

In [40]:
# prompt: ### F1. Explain the error, propose a fix
# **Prompt:**
# ```
# I ran this BigQuery SQL and got an error:
# SQL:-- Cost control note: For exploration, consider adding a WHERE clause to limit the date range, e.g.,
# -- WHERE Order_Date >= '2023-01-01' AND Order_Date < '2024-01-01'
# WITH monthly_revenue AS (
#   SELECT
#     DATE_TRUNC(Order_Date, MONTH) AS month,
#     SUM(Sales) AS monthly_revenue
#   FROM
#     `original-wonder-471819-n2.lab1_foundation.superstore_clean`
#   WHERE
#     Segment = 'Corporate' -- Filter for the 'Corporate' segment
#   GROUP BY
#     month
# )
# SELECT
#   month,
#   monthly_revenue,
#   AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS ma_3
# FROM
#   monthly_revenue
# ORDER BY
#   month ASC;
# Error:❌ An error occurred: 400 No matching signature for aggregate function AVG
#   Argument types: STRUCT<month DATE, monthly_revenue FLOAT64>
# Act as a BigQuery trouble‑shooter.
# 1) Identify the root cause.
# 2) Propose the smallest possible fix.
# 3) Suggest a quick sanity check query to verify the fix. Return only the corrected SQL and a 2‑sentence rationale.

print("✅ Step 1: Defining the query string with corrected AVG function usage...")

query_string = """
-- Cost control note: For exploration, consider adding a WHERE clause to limit the date range, e.g.,
-- WHERE Order_Date >= '2023-01-01' AND Order_Date < '2024-01-01'
WITH monthly_revenue AS (
  SELECT
    DATE_TRUNC(Order_Date, MONTH) AS month,
    SUM(Sales) AS monthly_revenue
  FROM
    `original-wonder-471819-n2.lab1_foundation.superstore_clean`
  WHERE
    Segment = 'Corporate' -- Filter for the 'Corporate' segment
  GROUP BY
    month
)
SELECT
  month,
  monthly_revenue,
  -- The AVG function was applied to the entire row (STRUCT) instead of the monthly_revenue column.
  -- Corrected to apply AVG to the 'monthly_revenue' column.
  AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS ma_3
FROM
  monthly_revenue
ORDER BY
  month ASC;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your table exists and has data for the 'Corporate' segment.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

print("\n--- Rationale ---")
print("The error 'No matching signature for aggregate function AVG' occurred because the AVG function was attempting to aggregate a STRUCT (the entire row) instead of a numeric column. The fix is to explicitly apply AVG to the 'monthly_revenue' column.")
print("Sanity Check: Verify that the 'ma_3' column correctly calculates the average of the current month's revenue and the two preceding months' revenues.")


### F2. Reduce cost / improve speed
**Prompt:**
```
Act as a BigQuery cost optimizer.
Given this query (below), list 3 ways to reduce scanned bytes and improve performance without changing the business logic.
[PASTE YOUR SQL HERE]
Prioritize: partition filters, column pruning, pre-aggregations, and temporary results via CTEs.
```

In [ ]:
# prompt: Act as a BigQuery cost optimizer.
# Given this query (below), list 3 ways to reduce scanned bytes and improve performance without changing the business logic.
# query_string = """
# SELECT
#   Sub_Category,
#   SUM(Profit) AS total_profit
# FROM
#   `original-wonder-471819-n2.lab1_foundation.superstore_clean`
# GROUP BY
#   Sub_Category
# HAVING
#   SUM(Profit) < 0
# ORDER BY
#   total_profit ASC;
# """
# Prioritize: partition filters, column pruning, pre-aggregations, and temporary results via CTEs.

print("✅ Step 1: Analyzing the provided query for optimization opportunities...")

# The provided query is:
# SELECT
#   Sub_Category,
#   SUM(Profit) AS total_profit
# FROM
#   `original-wonder-471819-n2.lab1_foundation.superstore_clean`
# GROUP BY
#   Sub_Category
# HAVING
#   SUM(Profit) < 0
# ORDER BY
#   total_profit ASC;

print("\n--- Optimization Strategies ---")

print("\n1. Column Pruning:")
print("   - The query only needs `Sub_Category` and `Profit`. By selecting only these columns in the initial scan (or in a CTE), BigQuery can avoid reading unnecessary data.")
print("   - Example: `SELECT Sub_Category, Profit FROM ...`")

print("\n2. Partition Filter (if applicable):")
print("   - If the `superstore_clean` table is partitioned by a date column (e.g., `Order_Date`), adding a `WHERE` clause to filter for a specific date range can drastically reduce the amount of data scanned.")
print("   - Example: `WHERE Order_Date >= '2023-01-01' AND Order_Date < '2024-01-01'` (assuming `Order_Date` is a partition column).")

print("\n3. Pre-aggregation using a CTE:")
print("   - While the current query already aggregates, if there were more complex operations before the final aggregation, a CTE could be used to pre-aggregate intermediate results, potentially reducing the data processed in later stages.")
print("   - For this specific query, a CTE might not offer significant gains over the direct approach, but it's a general strategy for complex queries.")
print("   - Example (demonstrative, not strictly necessary for this simple query):")
print("""
   WITH subcat_profit AS (
     SELECT
       Sub_Category,
       SUM(Profit) AS total_profit
     FROM
       `original-wonder-471819-n2.lab1_foundation.superstore_clean`
     GROUP BY
       Sub_Category
   )
   SELECT
     Sub_Category,
     total_profit
   FROM
     subcat_profit
   HAVING
     total_profit < 0
   ORDER BY
     total_profit ASC;
   """)

print("\nNote: For this particular query, column pruning is the most straightforward and impactful optimization. Partition filtering is highly effective if the table is partitioned by date.")

# Since the task is to list the ways and not to rewrite the query with all optimizations applied at once,
# we will not execute a modified query here. The explanation above covers the requested optimizations.


## Part G — Validation & Counter‑examples (DIVE: Validate)
**Aim:** Avoid “first‑answer fallacy” by testing alternatives.

### G1. Ask for counter‑queries
**Prompt:**
```
I concluded that 'Tables' is a high‑sales but negative‑profit sub-category due to high discounts.
Create two alternative BigQuery SQL queries that could falsify or nuance this finding:
- One that slices by region and time
- One that controls for order priority or ship mode
Return BigQuery SQL only, then a one-paragraph note on how to compare outcomes.
```

In [41]:
# prompt: I concluded that 'Tables' is a high‑sales but negative‑profit sub-category due to high discounts.
# Create two alternative BigQuery SQL queries that could falsify or nuance this finding:
# - One that slices by region and time
# - One that controls for order priority or ship mode
# Return BigQuery SQL only, then a one-paragraph note on how to compare outcomes.

print("✅ Step 1: Defining the query strings for counter-examples...")

# Query 1: Slice by region and time
query_string_region_time = """
WITH monthly_sales_by_region AS (
  SELECT
    EXTRACT(YEAR FROM Order_Date) AS sales_year,
    EXTRACT(MONTH FROM Order_Date) AS sales_month,
    Region,
    Sub_Category,
    SUM(Sales) AS total_sales,
    SUM(Profit) AS total_profit
  FROM
    `original-wonder-471819-n2.lab1_foundation.superstore_clean`
  WHERE
    Sub_Category = 'Tables'
  GROUP BY
    sales_year,
    sales_month,
    Region,
    Sub_Category
)
SELECT
  sales_year,
  sales_month,
  Region,
  Sub_Category,
  total_sales,
  total_profit,
  -- Calculate profit margin to normalize for sales volume
  CASE
    WHEN total_sales = 0 THEN 0
    ELSE (total_profit / total_sales) * 100
  END AS profit_margin_pct
FROM
  monthly_sales_by_region
ORDER BY
  sales_year ASC,
  sales_month ASC,
  Region ASC;
"""

# Query 2: Control for order priority or ship mode
query_string_priority_shipmode = """
SELECT
  Sub_Category,
  Ship_Mode,
  SUM(Sales) AS total_sales,
  SUM(Profit) AS total_profit,
  -- Calculate profit margin to normalize for sales volume
  CASE
    WHEN SUM(Sales) = 0 THEN 0
    ELSE (SUM(Profit) / SUM(Sales)) * 100
  END AS profit_margin_pct
FROM
  `original-wonder-471819-n2.lab1_foundation.superstore_clean`
WHERE
  Sub_Category = 'Tables'
GROUP BY
  Sub_Category,
  Ship_Mode
ORDER BY
  Ship_Mode ASC;
"""

print("✅ Step 2: Executing the first query (slice by region and time)...")
try:
    query_job1 = client.query(query_string_region_time)
    results_df1 = query_job1.to_dataframe()
    print(f"✅ Query 1 finished. Found {len(results_df1)} rows.")
    if not results_df1.empty:
        print("\n--- Results for Query 1 (Region and Time Slice) ---")
        display(results_df1.head())
    else:
        print("\n⚠️ Query 1 returned an empty result.")
except Exception as e:
    print(f"\n❌ An error occurred during Query 1 execution: {e}")

print("\n✅ Step 3: Executing the second query (control for ship mode)...")
try:
    query_job2 = client.query(query_string_priority_shipmode)
    results_df2 = query_job2.to_dataframe()
    print(f"✅ Query 2 finished. Found {len(results_df2)} rows.")
    if not results_df2.empty:
        print("\n--- Results for Query 2 (Ship Mode Control) ---")
        display(results_df2.head())
    else:
        print("\n⚠️ Query 2 returned an empty result.")
except Exception as e:
    print(f"\n❌ An error occurred during Query 2 execution: {e}")

print("\n--- Comparison Note ---")
print("Comparing the results of these two queries against the initial finding can nuance the conclusion. If the first query shows that 'Tables' are only negative profit in specific regions or months, it suggests a localized issue. If the second query reveals that negative profit is concentrated in certain `Ship_Mode`s, it points to a specific operational factor. By examining the `profit_margin_pct` in both, we can see if the negative profit is a consistent issue across sales volumes or if it's driven by specific conditions, potentially indicating that the initial conclusion about discounts might be incomplete or that other factors are at play.")


✅ Step 1: Defining the query strings for counter-examples...
✅ Step 2: Executing the first query (slice by region and time)...
✅ Query 1 finished. Found 140 rows.

--- Results for Query 1 (Region and Time Slice) ---


,sales_year,sales_month,Region,Sub_Category,total_sales,total_profit,profit_margin_pct
0,2014,1,West,Tables,333.000,-16.6500,-5.000000
1,2014,2,South,Tables,1256.220,75.3732,6.000000
2,2014,3,Central,Tables,2452.070,-89.1204,-3.634497
3,2014,3,East,Tables,3595.818,-1267.2787,-35.243127
4,2014,3,West,Tables,626.352,-23.4882,-3.750000



✅ Step 3: Executing the second query (control for ship mode)...
✅ Query 2 finished. Found 4 rows.

--- Results for Query 2 (Ship Mode Control) ---


,Sub_Category,Ship_Mode,total_sales,total_profit,profit_margin_pct
0,Tables,First Class,28800.7760,-1365.3665,-4.740728
1,Tables,Same Day,9644.3470,-1129.4225,-11.710720
2,Tables,Second Class,43693.7475,-3320.6799,-7.599897
3,Tables,Standard Class,124826.6615,-11910.0122,-9.541241



--- Comparison Note ---
Comparing the results of these two queries against the initial finding can nuance the conclusion. If the first query shows that 'Tables' are only negative profit in specific regions or months, it suggests a localized issue. If the second query reveals that negative profit is concentrated in certain `Ship_Mode`s, it points to a specific operational factor. By examining the `profit_margin_pct` in both, we can see if the negative profit is a consistent issue across sales volumes or if it's driven by specific conditions, potentially indicating that the initial conclusion about discounts might be incomplete or that other factors are at play.


## Part H — Synthesis (DIVE: Extend)
**Aim:** Turn analysis into business‑ready insights.

### H1. Executive‑style summary
**Prompt:**
```
Act as a business strategist.
Based on the following metrics/figures (briefly summarize your results here), write a 4-sentence executive summary:
- 1 sentence: what changed and by how much
- 1 sentence: why it likely changed (drivers)
- 1 sentence: recommended action (who/what/when)
- 1 sentence: metric to monitor next
```

In [42]:
# prompt: Act as a business strategist.
# Based on the following metrics/figures (briefly summarize your results here), write a 4-sentence executive summary:
# - 1 sentence: what changed and by how much
# - 1 sentence: why it likely changed (drivers)
# - 1 sentence: recommended action (who/what/when)
# - 1 sentence: metric to monitor next

print("✅ Step 1: Defining the executive summary based on hypothetical findings...")

# This is a template. You would replace the bracketed placeholders
# with actual findings from your previous BigQuery analyses.

executive_summary = """
Overall sales revenue saw a significant increase of 15% in the last quarter, primarily driven by strong performance in the 'Electronics' category and a successful promotional campaign. To capitalize on this momentum, we recommend expanding the marketing budget for 'Electronics' and similar high-performing categories in the upcoming fiscal year. We should closely monitor customer acquisition cost (CAC) and average order value (AOV) to ensure sustained profitable growth.
"""

executive_summary


✅ Step 1: Defining the executive summary based on hypothetical findings...


"\nOverall sales revenue saw a significant increase of 15% in the last quarter, primarily driven by strong performance in the 'Electronics' category and a successful promotional campaign. To capitalize on this momentum, we recommend expanding the marketing budget for 'Electronics' and similar high-performing categories in the upcoming fiscal year. We should closely monitor customer acquisition cost (CAC) and average order value (AOV) to ensure sustained profitable growth.\n"

### H2. Convert final SQL into an automated job (optional)
**Prompt (use only after your SQL is final):**
```
Convert my final BigQuery SQL into a Python script that can run as a scheduled job from Colab or Cloud Functions.
Requirements:
- Use python‑bigquery client
- Parameterize date range
- Write results to a destination table `[YOUR_PROJECT].analytics.outputs_kpi`
- Add basic error handling & logging
Return one complete runnable script.
```

In [ ]:
# prompt: Convert my final BigQuery SQL into a Python script that can run as a scheduled job from Colab or Cloud Functions.
# Requirements:
# - Use python‑bigquery client
# - Parameterize date range
# - Write results to a destination table `[YOUR_PROJECT].analytics.outputs_kpi`
# - Add basic error handling & logging
# Return one complete runnable script.

from google.cloud import bigquery
from google.colab import auth
import logging
from datetime import datetime, timedelta

# Authenticate and set up logging
auth.authenticate_user()
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Configuration ---
PROJECT_ID = 'original-wonder-471819-n2'  # Replace with your project ID
DATASET_ID = 'lab1_foundation'         # Replace with your dataset ID
TABLE_ID = 'superstore_clean'          # Replace with your table ID
OUTPUT_DATASET_ID = 'analytics'        # Replace with your output dataset ID
OUTPUT_TABLE_ID = 'outputs_kpi'        # Replace with your output table ID

# Define the date range for the job (e.g., last month)
# You can adjust these to parameterize based on your scheduling needs
end_date = datetime.now().replace(day=1) - timedelta(days=1)
start_date = end_date.replace(day=1)

# --- BigQuery Client Initialization ---
client = bigquery.Client(project=PROJECT_ID)
output_table_ref = client.dataset(OUTPUT_DATASET_ID).table(OUTPUT_TABLE_ID)

# --- SQL Query Definition ---
# This is a placeholder. Replace with your FINAL BigQuery SQL query.
# Ensure it's parameterized for dates if needed.
# Example: Using a simplified version of a previous query for demonstration.
# The query should select columns that match the expected schema of the output table.
query_template = f"""
SELECT
  Sub_Category,
  SUM(Sales) AS total_sales,
  SUM(Profit) AS total_profit,
  -- Example of a calculated metric
  CASE
    WHEN SUM(Sales) = 0 THEN 0
    ELSE (SUM(Profit) / SUM(Sales)) * 100
  END AS profit_margin_pct
FROM
  `{{project_id}}.{{dataset_id}}.{{table_id}}`
WHERE
  Order_Date >= '{{start_date}}' AND Order_Date <= '{{end_date}}'
GROUP BY
  Sub_Category
ORDER BY
  total_sales DESC;
"""

# --- Job Execution Function ---
def run_bq_job():
    """
    Runs the BigQuery job to process data and write results to an output table.
    """
    logging.info(f"Starting BigQuery job for date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

    # Format the query with actual values
    formatted_query = query_template.format(
        project_id=PROJECT_ID,
        dataset_id=DATASET_ID,
        table_id=TABLE_ID,
        start_date=start_date.strftime('%Y-%m-%d'),
        end_date=end_date.strftime('%Y-%m-%d')
    )

    job_config = bigquery.QueryJobConfig(
        destination=output_table_ref,
        write_disposition="WRITE_APPEND",  # Or "WRITE_TRUNCATE" if you want to overwrite
        use_legacy_sql=False
    )

    try:
        logging.info("Submitting BigQuery query job...")
        query_job = client.query(formatted_query, job_config=job_config)

        # Wait for the job to complete
        query_job.result()
        logging.info(f"BigQuery job completed successfully. Results written to {PROJECT_ID}.{OUTPUT_DATASET_ID}.{OUTPUT_TABLE_ID}")

        # Optional: Log the number of rows written (requires fetching job statistics or querying the output table)
        # For simplicity, we'll just confirm job completion.

    except Exception as e:
        logging.error(f"An error occurred during BigQuery job execution: {e}")
        # Depending on your needs, you might want to raise the exception or handle it differently

# --- Main Execution Block ---
if __name__ == "__main__":
    # Ensure the output dataset and table exist (optional, but good practice)
    # You might want to create these manually or add logic here to create them if they don't exist.
    # For this example, we assume they exist.

    run_bq_job()

---
## Submission checklist
- [ ] Kept prompts precise and reproducible  
- [ ] Captured at least **one** CTE query and **one** window function query  
- [ ] Documented **two** validation attempts (counter‑queries or alternate slice)  
- [ ] Wrote a 4‑sentence executive summary based on results  
- [ ] (Optional) Converted final query into a scheduled job
---